# TP2 - Realidade Aumentada

__Aluno:__ Vinicius Silva Gomes

__Matrícula:__ 2021421869

__Link do vídeo:__ [https://www.youtube.com/watch?v=cS0NltWVnqo&ab_channel=ViniciusGomes](https://www.youtube.com/watch?v=cS0NltWVnqo&ab_channel=ViniciusGomes)

In [1]:
# Importa as bibliotecas necessárias para o programa

import numpy as np
import cv2
import os

import sys
import PIL
import pygame as pg

from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *

from PIL import Image

from objloader import *

print('NumPy:', np.__version__)
print('OpenCV:', cv2.__version__)
print('OpenGL:', OpenGL.__version__)
print('PIL:', PIL.__version__)
print('Python:', sys.version)
print('PyGame:', pg.__version__)

pygame 2.1.2 (SDL 2.0.16, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html
NumPy: 1.23.2
OpenCV: 4.6.0
OpenGL: 3.1.6
PIL: 9.0.1
Python: 3.10.6 (main, Nov  2 2022, 18:53:38) [GCC 11.3.0]
PyGame: 2.1.2


## Extraindo frames do vídeo para calibração

Esse pequeno script foi feito para extrair os frames do vídeo e usá-los para calibrar a câmera e obter os parâmetros intrínsecos da câmera. O script, em suma, carrega o vídeo, usando o OpenCV, e salva cada frame no disco, em uma pasta chamada ./fames. O script facilitou a extração dos frames e tornou o processo mais automático, por isso ele foi utilizado.

Dessa pasta, foram escolhidos 5 frames que apresentavam angulações, distâncias e rotações diferentes do tabuleiro xadrez e esses frames foram separados para serem usados na calibração da câmera. Esses frames escolhidos foram: __frame0.jpg, frame161.jpg, frame260.jpg, frame702.jpg e frame800.jpg__. 

__OBS: Para que as imagens sejam salvas, a pasta ./frames precisa ter sido criada previamente.__

In [2]:
# Extrai os frames do vídeo para realizar a calibração da câmera

cam = cv2.VideoCapture("./entrada.mp4")

current_frame = 0

while(True):
    ret, frame = cam.read()

    if ret:
        # Para esse trecho funcionar uma pasta ./frames deve existir no diretório do notebook
        name = './frames/frame' + str(current_frame) + '.jpg'
  
        cv2.imwrite(name, frame)
  
        current_frame += 1
    else:
        break

cam.release()
cv2.destroyAllWindows()

## Obtendo os parâmetros intrínsecos e os coeficientes de distorção

Com os frames selecionados, foi usado o __MatLAB__ para obter os parâmetros intrísecos da câmera. A opção "Camera Calibration" foi a escolhida. Os pontos do tabuleiro foram selecionados e, após o mapeamento e as devidas funções internas do MatLAB terem sido executadas, a matriz de parâmetros intrínsecos foi obtida. Além disso, a partir dessa calibração os coeficientes de distorção também puderam ser obtidos. O MatLAB foi escolhido apenas por praticidade, afinal, softwares como o __Octave__ também seriam completamente adequados para realizar essa tarefa (a função __cv2::calibrateCamera()__ da OpenCV também poderia ser utilizada mas, por orientação do professor, a calibração foi realizada extra-código, no MatLAB).

A próxima célula apresenta a declaração dessas matrizes com os dados de output do MatLAB.

__OBS: A matriz informada pelo MatLAB é transposta da matriz utilizada nesse Notebook. Ela foi transposta para se parecer mais com a matriz apresentada durante as aulas e poder ser utilizada corretamente pelas funções da OpenCV e OpenGL.__

In [3]:
# Matriz com os parâmetros intrínsecos da câmera
intrinsic_params = np.array(([410.556385620329, 0, 313.422287942592],
                             [0, 409.988094875811, 233.665786823660],
                             [0, 0, 1]), dtype="float32")

# Vetor com os coeficientes de distorção obtidos na calibração
distortion_coefficients = np.array([0.094979724334344, -0.258369409395469, 0, 0], dtype="float32")

print(intrinsic_params)
print(distortion_coefficients)

[[410.5564    0.      313.4223 ]
 [  0.      409.9881  233.66579]
 [  0.        0.        1.     ]]
[ 0.09497973 -0.25836942  0.          0.        ]


## Localizando os alvos ao longo do vídeo

A célula a seguir apresenta funções que são utilizadas na identificação dos alvos ao longo do frame. Estão descritas algumas funções intermerdiárias/auxiliares, como a __normalized_cross_correlation()__ e a __compare_target()__; juntamente com a função principal, que retorna o frame no qual os alvos identificados estão contornados de azul claro, as coordenadas de todos os alvos no frame e a orientação de cada um desses alvos. A seguir, cada função será melhor explicada/especificada: 

#### __normalized_cross_correlation():__
Recebe um possível alvo retirado de um frame do vídeo e o compara com o template original do alvo pelo método da Correlação Cruzada Normalizada.

O resultado é um valor em ponto flutuante entre 0 e 1 que indica a relação entre ambas as imagens: 1 elas são muito parecidas e 0 elas não são nada parecidas. Qualquer função que mede similaridade entre imagens poderia ser usada, como o RMSE ou MAE. Portanto, a escolha da função foi completamente arbitrária, visto que todas essas funções foram discutidas durante as aulas e estariam aptas a serem usadas.

#### __find_orientation():__
Recebe o possível alvo retificado e o template original do alvo.

Com isso, a função rotaciona o alvo em todas as direções possíveis e calcula a __Correlação Cruzada Normalizada__, através da função __normalized_cross_correlation()__, para cada uma das direções e escolhe aquela que resulte no maior coeficiente (mais semelhante). Se esse valor estiver entre __0.65 e 1__, limiares definidos de maneira arbitrária, a partir de alguns testes com o vídeo; a função retorna __True__ para alvo e retorna qual a orientação desse alvo: __0 -> aponta para Cima__, __1 -> aponta para a Direita__, __2 -> aponta para Baixo__ e __3 -> aponta para a Esquerda__. Caso não seja um alvo, a função retorna __False__ e ã direção __-1__, indicando que esse polígono não se encaixa como um alvo na imagem.

#### __compare_target():__
Recebe como parâmetros o frame binarizado, as 4 extremidades do possível alvo e o frame original.

Essa função é responsável por calcular a homografia, através da função __cv2::findHomography()__, do possível alvo, passando esse objeto para o mesmo plano do template original do alvo, que é carregado do diretório por essa função. Após isso, uma transformação perspectiva é feita, com a função __cv2::warpPerspective()__, usando como base a matriz de homografia calculada no passo anterior, e, com isso, a imagem do possível alvo estará retificada e poderá ser comparada com o template original. Após isso, a função __find_orientation()__ será chamada, passando o possível alvo retificado e o template, e seu resultado é retornado pela função __compare_target()__.

Seria possível fazer essa reitificação a partir de outros métodos ou usar outras funções da OpenCV, mas como a matriz de homografia foi uma das formas vistas em sala de aula e seu funcionamento foi bem discutido, achei conveniente retificar o conjunto de pontos a partir dela.

#### __identify_targets():__
É a função principal da etapa de detecção dos alvos no frame. Só recebe o frame original do vídeo como entrada e processa todas as informações necessárias.

A abordagem usada para identificar os alvos será a partir da identificação das bordas no frame. Dessa forma, com as bordas enfatizadas, a função que identifica contornos age de maneira mais efetiva. Depois disso, basta filtrar bem os contornos e testá-los, para comparar se são alvos ou não. Os que derem match com o template original serão enviados para um vetor separado, junto com sua orientação, e, ao final, a função retorna esses marcadores e o frame original do vídeo com os alvos contornados de azul claro.

Para tanto, primeiramente, o frame é convertido para tons de cinza e é binarizado pela função __cv2::threshold()__. Ele será binarizado pois facilita a identificação de suas bordas e, com isso, a identificação dos contornos dos alvos. A função __cv2::findContours()__, então, é usada para identificar os contornos a partir das bordas enfatizadas. Para filtrar um pouco quais contornos obtidos serão analisados, o formato de suas arestas é aproximado, pela função __cv2::approxPolyDPapproxPolyDP()__, e o perímetro de cada polígono obtido é calculado, pela função __cv2::arcLength()__. Caso o polígono tenha 4 vértices e perímetro entre 140 e 450, significa que ele potencialmente é um alvo no frame. Então, esse possível alvo vai ser enviado para as funções de comparação e, caso ele realmente seja um alvo, ele é inserido no vetor __target_contours__, que contém os contornos a serem traçados na imagem, e tem o formato de seu array manipulado para ser usado pela função de renderização 3D, sendo inserido no vetor __targets__ após a formatação.

In [4]:
# Calcula a Correlação Cruzada Normalizada entre um possível alvo na cena e o template original a ser identificado
def normalized_cross_correlation(target, template):
    np.seterr(invalid='ignore')
    
    target_normalized = (target - target.mean()) / np.std(target)
    template_normalized = (template - template.mean()) / np.std(template)
    
    return np.mean(target_normalized * template_normalized)

# Dado o possível alvo retificado, calcula sua semelhança com todas as possíveis rotações do
# template e retorna, primeiramente, se o possível alvo é de fato um alvo e qual a sua orientação, caso seja.
def find_orientation(rectified, template):
    # Rotaciona o template nas direções possíveis
    up = template.copy()
    right = cv2.rotate(template, cv2.ROTATE_90_CLOCKWISE)
    down = cv2.rotate(template, cv2.ROTATE_180)
    left = cv2.rotate(template, cv2.ROTATE_90_COUNTERCLOCKWISE)
        
    # Calcula a similaridade para cada uma delas
    similarity_up = normalized_cross_correlation(rectified, up)
    similarity_right = normalized_cross_correlation(rectified, right)    
    similarity_down = normalized_cross_correlation(rectified, down)
    similarity_left = normalized_cross_correlation(rectified, left)
    
    similarities = [similarity_up, similarity_right, similarity_down, similarity_left]
    
    # Identifica a maior similaridade obtida
    max_similarity = max(similarities)
    max_index = similarities.index(max_similarity)
    
    # Caso a similaridade esteja no limiar estabelecido, o alvo foi identificado e terá sua orientação retornada
    # 0 -> cima, 1 -> direita, 2 -> baixo, 3 -> esquerda
    if max_similarity > 0.65 and max_similarity < 1:
        return True, max_index
    else:
        return False, -1
    
# Dado o frame com um possível alvo e as coordenadas do alvo, retifica a imagem para saber se é um alvo ou não 
def compare_target(thresh, target, copy):
    template = cv2.imread("./alvo.jpg", 0)
    
    # Cria a matriz com os pontos do template
    template_points = np.array([[0, 0],
                            [template.shape[0], 0],
                            [template.shape[0], template.shape[1]],
                            [0, template.shape[1]]])
    
    # Calcula a homografia do possível alvo em relação a matriz dos pontos do template
    homography, _ = cv2.findHomography(target, template_points)
    
    # Retifica o alvo com a matriz de homografia
    rectified = cv2.warpPerspective(thresh, homography, (template.shape[0], template.shape[1]))
        
    # Obtém se ele é um alvo e qual a orientação dele, caso seja
    is_target, orientation = find_orientation(rectified, template)
    
    return is_target, orientation
    
# Função principal que usa todas as outras para identificar os alvos em cada frame
def identify_targets(frame):
    copy = frame.copy()
    # Converte o frame para escala de tons de cinza
    gray_frame = cv2.cvtColor(copy, cv2.COLOR_BGR2GRAY)
        
    # Calcula a binarização do frame
    _, thresh = cv2.threshold(gray_frame, 127, 255, cv2.THRESH_BINARY)
        
    # Enfatiza as bordas presentes no frame binarizado
    edged = cv2.Canny(thresh, 80, 160)
    
    # Cria os contornos ao longo das bordas do frame
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
    target_contours = []
    targets = []

    # Para cada contorno identificado
    for contour in contours:
        # Aproxima sua forma geométrica para obter o número de lados
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, perimeter * 0.1, True)
        no_vertices = len(approx)
        
        # Caso o contorno identificado tenha 4 lados e seja convexo
        if no_vertices == 4 and perimeter >= 140 and perimeter <= 450:
            # Verifica se ele é um alvo
            is_target, orientation = compare_target(thresh, approx, copy)
                        
            # Caso seja um alvo, insere no vetor de alvos identificados naquele frame
            if is_target:
                target_contours.append(approx)
                targets.append((approx[:, 0][:].astype("float32"), orientation))
                
    # Desenha de azul os contornos identificados no frame
    cv2.drawContours(copy, target_contours, -1, (255, 255, 0), 3)
    
    # Retorna o frame contornado e os alvos identificados nesse frame
    return copy, targets

## Funções da OpenGL

A célula a seguir contém a implementação das principais funções usadas pela OpenGL. Estão definidas desde funções auxiliares executadas pelo código até as funções principais da OpenGL, que definem callbacks e outros aspectos importantes para a configuração correta da máquina de estados da OpenGL.

#### __initOpenGL():__
Recebe os parâmetros da câmera e as diensões da janela e inicializa o cone de perspectiva da câmera principal do OpenGL.

#### __load_background():__
Recebe como parâmetro o frame original do vídeo.

Essa função vai gerar o ID para a textura do background e manipular o frame para que ele possa ser usado pela OpenGL como textura de um quadrado que vai representar o fundo do vídeo. Para isso, ela converte o sistema de cores da imagem, que é carregado pela OpenCV em BGR, para RGB usando a função __cv2::cvtColor()__, faz o flip na imagem no eixo X através da função __cv2::flip()__, para alinhar os eixos de coordenadas da OpenCV com a OpenGL. Depois disso, ela transforma a imagem texto, para que ela possa ser utilizada como textura pela OpenGL e define os filtros de escala da imagem, no caso, aproximação a partir dos vizinhos mais próximos. Por fim, a função retorna o ID gerado para o background e o background que foi transformado.

#### __place_background():__
É a função responsável por pegar o ID do background e o background propriamente na forma de string para inserí-lo como textura de fundo da janela. Para tal, um quadrado com projeção ortogonal e com as dimensões da janela é definido e a textura do background, que foi previamente processada, é vinculada a ele. Dessa forma, a imagem do frame aparece de fundo na janela.

#### __verify_upper_target():__
É a função responsável por identificar se o alvo passado como parâmetro é o alvo que aparece na parte de cima do vídeo. Essa verificação específica é feita para que seja possível fazer o alvo que fica mais acima girar no sentido horário, o contrário dos outros Pikachu's, que devem girar em sentido anti-horário.

Em suma, dois limiares para as coordenadas X e Y foram definidos. Esses limiares foram escolhidos de maneira empírica, a partir de testes, até que os valores se adequassem as configurações do vídeo. Caso alguma coordenada do alvo analisado esteja dentro desses limiares, significa que esse alvo provavelmente é o alvo de cima, então o Pikachu em cima desse alvo rotaciona para o sentido contrário ao dos outros.

#### __render_cube():__
Recebe o tamanho do cubo a ser desenhado e utiliza a função __glutWireCube()__ para desenhar um cubo com o tamanho desejado na posição atual da câmera virtual da OpenGL. Essa função vai ser chamada logo após o cálculo da matriz de projeção para inserir o Pikachu no vídeo, assim, o cubo será desenhado no mesmo lugar que o Pikachu.

#### __render_arrow():__
Recebe o tamanho da flecha a ser desenhada e a orientação do alvo em que ela estará em cima.

Dado esses parâmetros, o código estima para quais direções a linha e o cone serão transladados e qual será a rotação do cone para que a flecha seja devidamente montada na cena. A linha é criada com a função padrão da OpenGL para criar linhas e o cone é criado com a função __glutWireCone()__, da Glut. Assim como o cubo, a flecha será criada logo após a matriz de projeção que insere o Pikachu em cima do alvo ser criada, então ela será criada no centro do alvo e vai apontar para a direção que esse alvo está na cena.

#### __render_3D_objects():__
É a função principal da renderização 3D dos objetos. Recebe o objeto do Pikachu que foi carregado na função __displayCallback()__, os pontos do alvo e a orientação do alvo, os parâmetros intrínsecos da câmera e os coeficientes de distorção da câmera.

Os pontos no mundo vão ser criados e, com eles, será possível calcular os vetores de rotação e translação, através da função __cv2::solvePnP()__. Os vetores de rotação serão transformados na matriz de rotação a partir da transformação de rotação de Rodrigues, que é calculada pela função __cv2::Rodrigues()__. Com a matriz de rotação e o vetor de translação, é possível montar a matriz de projeção para o alvo em questão. Alguns coeficientes dessa matriz serão negados para inverter os eixos Y e Z da OpenCV para OpenGL. A matriz de projeção é transposta e carregada na máquina de estados da OpenGL. Após isso, o Pikachu é renderizado em tela e as funções que renderizam o cubo e a flecha também são chamadas para que esses objetos apareçam na cena. O Pikachu, no entanto, será renderizado por último, pois antes dele aparecer em tela a função __glRotatef()__ aplica uma pequena rotação no eixo do objeto. Dessa forma, o cubo e a flecha não giram, mas o Pikachu é capaz de girar em torno de seu próprio eixo a medida que os frames do vídeo vão atualizando.

#### __displayCallback():__
Função principal para renderização dos frames e objetos 3D na janela da OpenGL.

Carrega o objeto do Pikachu e busca um frame no vídeo. Se esse frame lido for válido (vídeo ainda não tiver sido finalizado), os alvos são identificados e o frame com os alvos contornados é passado para a função que o define como background da janela. Após isso, cada marcador é passado para a função que renderiza os objetos 3D na cena e o Pikachu, o cubo e a flecha são renderizados em cima de cada alvo. Com a renderização finalizada, a glut troca os buffers de imagem, com a função __glutSwapBuffers()__, e o próximo frame vai aparecer em tela.

#### __idleCallback():__
Chama a função da Glut que define que o frame atual precisa ser remontado, então a função de display é chamada novamente e um novo frame do vídeo vai ser colocado em tela. Além disso, o valor do ângulo de rotação do Pikachu é redefinido nesse callback. Assim, ele é capaz de rotacionar 6 graus a cada frame e, quando ultrapassar 360, volta para 0 e continua girando de maneira automática.

In [5]:
# Realiza os ajustes de perspectiva da câmera do OpenGL
def initOpenGL(intrinsic_params, dimensions):
    (width, height) = dimensions
    
    # Cor do fundo ao realizar o clear
    glClearColor(0.0, 0.0, 0.0, 0.0)
    glClearDepth(1.0)

    glEnable(GL_DEPTH_TEST)

    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    
    fx = intrinsic_params[0, 0];
    fy = intrinsic_params[1, 1];
    
    # Calcula o fovy e o aspect ratio e determina as distâncias 'near' e 'far' da câmera
    fovy = 2 * np.arctan(0.5 * height/fy) * 180/np.pi;
    aspect = (width * fy)/(height * fx);
    near = 0.1;
    far = 100.0;
    
    # Especifica o cone de visão da câmera
    gluPerspective(fovy, aspect, near, far);

# Gera o ID para a textura do background e manipula o background para ser usado como textura pela OpenGL
def load_background(frame):
    # Gera o ID para o background e o vincula a uma textura
    background_id = glGenTextures(1)
    glBindTexture(GL_TEXTURE_2D, background_id)
    
    # Converte o modo de cor OpenCV -> OpenGL e rotaciona os eixos
    background = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    background = cv2.flip(background, 0)
    
    # Transforma a imagem para bytes, para ser usada como textura pela OpenGL
    height, width, channels = background.shape
    background = np.frombuffer(background.tobytes(), dtype=background.dtype, count = height * width * channels)    
    background.shape = (height, width, channels)

    # Cria a textura do background na OpenGL
    glTexParameterf(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameterf(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, width, height, 0, GL_RGB, GL_UNSIGNED_BYTE, background)
    
    # Retorna o ID do background e o background propriamente
    return background_id, background

# Cria um quadrado do tamanho da janela e projeta nele a textura do background
def place_background(frame, dimensions):
    # Pega o ID da textura e a textura do frame atual
    background_id, background = load_background(frame)
    
    (width, height) = dimensions
    
    # Define uma projeção ortográfica com as dimensões iguais as da janela da OpenGL
    glDepthMask(GL_FALSE)
    glMatrixMode(GL_PROJECTION)
    glPushMatrix()
    glLoadIdentity()
    gluOrtho2D(0, width, 0, height)
    
    # Vincula o ID da textura ao background e o carrega como uma textura de texto
    glMatrixMode(GL_MODELVIEW)
    glBindTexture(GL_TEXTURE_2D, background_id)
    glTexImage2D(GL_TEXTURE_2D, 0, 3, width, height, 0, GL_RGB, GL_UNSIGNED_BYTE, background)
    glPushMatrix()

    # Cria os pontos de projeção do quadrado/background
    glBegin(GL_QUADS)
    glTexCoord2i(0, 0); glVertex2i(0, 0)
    glTexCoord2i(1, 0); glVertex2i(width, 0)
    glTexCoord2i(1, 1); glVertex2i(width, height)
    glTexCoord2i(0, 1); glVertex2i(0, height)
    glEnd()
    
    glPopMatrix()
    
    glMatrixMode(GL_PROJECTION)
    glPopMatrix()
    
    # Volta o modo de matriz para MODELVIEW, disabilita a textura 2D e ativa a máscara de profundidade ("reseta" as configurações para o próximo frame)
    glMatrixMode(GL_MODELVIEW)
    glDepthMask(GL_TRUE)
    glDisable(GL_TEXTURE_2D)
    glFlush()

# Verifica se o alvo selecionado é o alvo mais acima presente no vídeo (alvo que gira no sentido horário)
def verify_upper_target(target):
    # Limiares setados a partir de testes executando o código
    x_limit = 310
    y_limit = 210
    
    # Se o alvo tem alguma coordenada que ultrapasse esse limiar, significa que é o alvo que gira no sentido horário
    for coord in target:
        if coord[0] >= x_limit and coord[1] <= y_limit:
            return True
        
    return False

# Desenha o cubo na cena com auxilio da Glut
def render_cube(length):
    glutWireCube(length);

# Desenha a flecha com a direção do alvo, tendo como centro o próprio alvo
def render_arrow(length, orientation):
    x_dir = 0
    y_dir = 0

    x_cone = 1
    y_cone = 0
    
    # Série de IF's que verificam a orientação do alvo e configuram os parâmetros
    # para translação e rotação da linha e cone que formam a flecha com direção do alvo
    if orientation == 0:
        x_dir = 0
        y_dir = -length
        x_cone = 1
        y_cone = 0
        
    if orientation == 1:
        x_dir = -length
        y_dir = 0
        x_cone = 0
        y_cone = -1
        
    if orientation == 2:
        x_dir = 0
        y_dir = length
        x_cone = -1
        y_cone = 0
        
    if orientation == 3:
        x_dir = length
        y_dir = 0
        x_cone = 0
        y_cone = 1
    
    glPushMatrix()
    
    # Desenha a linha da flecha com a direção do alvo
    glBegin(GL_LINES)
    glVertex3d(0, 0, 0)
    glVertex3d(x_dir, y_dir, 0)
    glEnd()
    
    # Desenha e rotaciona a ponta da flecha com a direção do alvo
    glTranslated(x_dir, y_dir, 0)
    glRotated(90, x_cone, y_cone, 0)
    glutWireCone(0.4, 1.4, 10, 10)
    
    glPopMatrix()
    
# Renderiza o Pikachu, o cubo e a flecha com a direção do alvo na cena
def render_3D_objects(obj, target_points, orientation, intrinsic_params, distortion_coefficients):
    # Pontos no mundo definidos de maneira arbitrária
    object_points = np.array([[1, -1, 0], [-1, -1, 0],
                              [-1, 1, 0], [1, 1, 0]], dtype="float32")
    
    # Calcula os vetores de rotação e translação
    _, rotation_vec, translation_vec = cv2.solvePnP(object_points, target_points, intrinsic_params, distortion_coefficients)
     
    # Converte o vetor de rotação para uma matriz de rotação 
    rotation_matrix = cv2.Rodrigues(rotation_vec)[0]
    
    # Monta a matriz de projeção
    # Os coeficientes da segunda e terceira linha foram negados para inverter os eixos Y e Z, para passar
    # do sistema de coordenadas da OpenCV para OpenGL
    projection_matrix = np.array([[rotation_matrix[0, 0], rotation_matrix[0, 1], rotation_matrix[0, 2], translation_vec[0]],
                                [-rotation_matrix[1, 0], -rotation_matrix[1, 1], -rotation_matrix[1, 2], -translation_vec[1]],
                                [-rotation_matrix[2, 0], -rotation_matrix[2, 1], -rotation_matrix[2, 2], -translation_vec[2]],
                                [0, 0, 0, 1]], dtype="float32")
    
    # Transpõe a matriz
    projection_matrix = projection_matrix.T
    
    glPushMatrix()
        
    # Carrega a matriz de projeção no OpenGL
    glLoadMatrixf(projection_matrix)
    
    # Renderiza um cubo ao redor do alvo
    render_cube(3)
    
    # Renderiza a flecha que aponta para a direção do alvo
    render_arrow(3, orientation)
    
    is_upper_target = verify_upper_target(target_points)
    
    # Carrega a rotação do Pikachu para aquele frame
    if is_upper_target:
        glRotatef(-angle, 0, 0, 1)
    else:
        glRotatef(angle, 0, 0, 1)
    
    # Renderiza o modelo do Pikachu
    glCallList(obj.gl_list)
    
    glPopMatrix()
    
def displayCallback():
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glEnable(GL_TEXTURE_2D)
    
    # Carrega o objeto do Pikachu
    obj = OBJ("Pikachu.obj", swapyz=True)
        
    # Lê um frame do vídeo
    ret, frame = video.read()
    
    if ret:
        # Identifica os alvos e pega o frame do vídeo com os alvos identificados contornados
        frame_with_contour, targets = identify_targets(frame)
                
        # Define esse frame como background da janela do OpenGL
        place_background(frame_with_contour, (640, 480))
                
        # Para cada target identificado
        for target in targets:
            target_coords, orientation = target
            
            # Renderiza os objetos 3D na cena (Pikachu, cubo e flecha direcionada)
            render_3D_objects(obj, target_coords, orientation, intrinsic_params, distortion_coefficients)
        
        # Troca o buffer da janela atual
        glutSwapBuffers()

def idleCallback():
    # Acessa a variável de rotação do Pikachu e altera seu valor
    global angle
    angle += 6
    
    # Caso ela passe dos 360 graus, retorna seu valor pra 0
    if angle > 360:
        angle = 0

    glutPostRedisplay()

## Main da OpenGL

Função principal do OpenGL. É a célula que cria a janela e executa as funções auxiliares e de callback definidas nas células anteriores. Essa célula vai carregar o vídeo de entrada do diretório, inicializar o OpenGL e a Glut e, por fim, criar uma janela com o título __"TP2 - Realidade Aumentada - Vinicius Silva Gomes"__ e executar o vídeo com os alvos contornados e os objetos 3D renderizados em cima de cada alvo.

In [6]:
dimensions = (640, 480)

# Parâmetros intrínsecos e coeficientes de distorção obtidos com a calibração
intrinsic_params = np.array(([410.556385620329, 0, 313.422287942592],
                             [0, 409.988094875811, 233.665786823660],
                             [0, 0, 1]), dtype="float32")

# Vetor com os coeficientes de distorção obtidos na calibração
distortion_coefficients = np.array([0.094979724334344, -0.258369409395469, 0, 0], dtype="float32")

# Carrega o vídeo da pasta
video = cv2.VideoCapture("./entrada.mp4")

# Variável que indica o ângulo de rotação do Pikachu
angle = 0

# Inicializa a glut
glutInit()
glutInitDisplayMode(GLUT_RGBA | GLUT_DOUBLE | GLUT_DEPTH)
glutSetOption(GLUT_ACTION_ON_WINDOW_CLOSE, GLUT_ACTION_CONTINUE_EXECUTION)

glutInitWindowSize(*dimensions)

# Cria a janela e inicializa o OpenGL com os parâmetros da câmera e as dimensões da janela
window = glutCreateWindow(b'TP2 - Realidade Aumentada - Vinicius Silva Gomes')
initOpenGL(intrinsic_params, dimensions)

# Chama as funções principais de display e entra no loop principal do OpenGL
glutDisplayFunc(displayCallback)
glutIdleFunc(idleCallback)

# Chama o loop principal do OpenGL
glutMainLoop()

# Desaloca o vídeo de entrada
video.release()

/tmp/ipykernel_6343/37168942.py:169: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  projection_matrix = np.array([[rotation_matrix[0, 0], rotation_matrix[0, 1], rotation_matrix[0, 2], translation_vec[0]],
